In [1]:
# 运行
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 运行
dir_path = "./images"

# 

i = 0




In [8]:
# 运行
for fn in os.listdir(dir_path):
#     print(fn)
    
    dir_i = dir_path + "/" + fn + "/"
    filenames = [dir_i + fn1 for fn1 in os.listdir(dir_i)]
    
#     print(filenames)
    
    if i==0 :
        filenames_all = filenames
        labels = [i] * len(filenames)
    else:
        filenames_all = filenames_all + filenames
        labels_i = [i] * len(filenames)
        labels = labels + labels_i
        
    i = i + 1

In [ ]:
filenames_all

In [ ]:
for filename, label in zip(filenames_all, labels):
    print(filename, "----", label)

In [ ]:
image = open('./images/n02085620-Chihuahua/n02085620_10074.jpg', 'rb').read()

In [5]:
# 运行
def write_TFRecord_file(filenames_all, labels, tfrecord_file):
    with tf.io.TFRecordWriter(tfrecord_file) as writer:
        for filename, label in zip(filenames_all, labels):
#             print(filename)
            image = open(filename, 'rb').read()
            
            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
            }
            
            example = tf.train.Example(features=tf.train.Features(feature=feature))
#             print(example.SerializeToString)
            writer.write(example.SerializeToString())

In [3]:
# 运行
# 图片数据处理
feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

def parse_example(example_string):
    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_jpeg(feature_dict['image'], channels=3)
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [224, 224]) / 255.0
    return feature_dict['image'], feature_dict['label']

In [4]:
# 运行
data_dir = './data/'
tfrecord_file = data_dir + 'dogs.tfrecords'

if not os.path.isfile(tfrecord_file):
    write_TFRecord_file(filenames_all, labels, tfrecord_file)
    print('write TFRecord file:', tfrecord_file)
else:
    print(tfrecord_file, 'already exists')

./data/dogs.tfrecords already exists


In [5]:
# 运行
def read_TFRecord_file(tfrecord_file):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_file)
    
    dataset = raw_dataset.map(parse_example)
    
    return dataset

In [6]:
# 运行
dataset = read_TFRecord_file(tfrecord_file)

In [ ]:
# 取数据查看
sub_dataset = dataset.take(3)

for x,y in sub_dataset:
    print(x.numpy())
    print(y.numpy())

## 不按tfrecord获取图片

In [ ]:
for fn in os.listdir(dir_path):
#     print(fn)
    
    dir_i = dir_path + "/" + fn + "/"
    filenames = tf.constant([dir_i + fn1 for fn1 in os.listdir(dir_i)])
    
#     print(filenames)
    
    if i==0 :
        filenames_all = tf.constant(filenames)   
        labels = tf.zeros(filenames.shape, dtype=tf.int32)
    else:
        filenames_all = tf.concat([filenames_all, filenames], axis=-1)
        labels_i = tf.fill(filenames.shape, i)
        labels = tf.concat([labels, labels_i], axis=-1)
        
    i = i + 1
    
# dataset = tf.data.Dataset.from_tensor_slices((filenames_all, labels))

In [ ]:
def decode_image_and_resize(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image_resized = tf.image.resize(image_decoded, [224,224]) / 255.0
    return image_resized, label

In [ ]:
# dataset = dataset.map(map_func=decode_image_and_resize,
#                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.map(map_func=decode_image_and_resize,
                     num_parallel_calls=4)

# 数据集划分训练

In [9]:
# 运行
buffer_size = tf.size(labels)
buffer_size = tf.cast(buffer_size, dtype=tf.int64)
# labels.shape
# buffer_size
dataset = dataset.shuffle(buffer_size)

In [10]:
train_dataset = dataset.take(10000)

# for x,y in sub_dataset:
#     plt.title(y.numpy())
#     plt.imshow(x.numpy())
#     plt.show()

In [11]:
batch_size = 10
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
# dataset = dataset.batch(batch_size)
# dataset = dataset.prefetch(-1)

In [ ]:
def NASNetLarge_model(input_shape=(331, 331, 3)):
    nas = tf.keras.applications.nasnet.NASNetLarge(include_top=False,
                                       weights='imagenet',
                                       input_shape=input_shape)
    
    for layer in nas.layers:
        layer.trainable = False
        
    last = nas.output
    
    # 加入全连接层
    x = tf.keras.layers.Flatten()(last)
    
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.3)(x)
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.3)(x)
    #输出层
    x = tf.keras.layers.Dense(120, activation='softmax')(x)
    
    #模型建立
    model = tf.keras.models.Model(inputs=nas.input, outputs=x)
    
    model.summary()
    
    return model

In [13]:
def mobilenet_v2_model(input_shape=(224, 224, 3)):
    md = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False,
                                       weights='imagenet',
                                       input_shape=input_shape)
    
    for layer in md.layers:
        layer.trainable = False
        
    last = md.output
    
    # 加入全连接层
    x = tf.keras.layers.Flatten()(last)
    
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.3)(x)
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.3)(x)
    #输出层
    x = tf.keras.layers.Dense(120, activation='softmax')(x)
    
    #模型建立
    model = tf.keras.models.Model(inputs=md.input, outputs=x)
    
    model.summary()
    
    return model

In [12]:
def DenseNet121_model(input_shape=(224, 224, 3)):
    md = tf.keras.applications.densenet.DenseNet121(include_top=False,
                                       weights='imagenet',
                                       input_shape=input_shape)
    
    for layer in md.layers:
        layer.trainable = False
        
    last = md.output
    
    # 加入全连接层
    x = tf.keras.layers.Flatten()(last)
    
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.3)(x)
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.3)(x)
    #输出层
    x = tf.keras.layers.Dense(120, activation='softmax')(x)
    
    #模型建立
    model = tf.keras.models.Model(inputs=md.input, outputs=x)
    
    model.summary()
    
    return model

In [ ]:
def Xception_model(input_shape=(224, 224, 3)):
    md = tf.keras.applications.xception.Xception(include_top=False,
                                       weights='imagenet',
                                       input_shape=input_shape)
    
    for layer in md.layers:
        layer.trainable = False
        
    last = md.output
    
    # 加入全连接层
    x = tf.keras.layers.Flatten()(last)
    
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.3)(x)
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.3)(x)
    #输出层
    x = tf.keras.layers.Dense(120, activation='softmax')(x)
    
    #模型建立
    model = tf.keras.models.Model(inputs=md.input, outputs=x)
    
    model.summary()
    
    return model

In [13]:
# model = NASNetLarge_model()
# model = mobilenet_v2_model()
model = DenseNet121_model()
# model = Xception_model()
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [14]:
training_epochs = 5
# sub_dataset
train_dataset

<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>

In [15]:
history = model.fit(train_dataset, epochs=training_epochs, verbose=1)

Epoch 1/5
1000/1000 [==============================] - 3519s 4s/step - loss: 20.5856 - accuracy: 0.3830
Epoch 2/5
1000/1000 [==============================] - 4280s 4s/step - loss: 14.6497 - accuracy: 0.6349
Epoch 3/5
1000/1000 [==============================] - 4446s 4s/step - loss: 12.3544 - accuracy: 0.7119
Epoch 4/5
1000/1000 [==============================] - 4481s 4s/step - loss: 10.9166 - accuracy: 0.7559
Epoch 5/5
1000/1000 [==============================] - 4655s 5s/step - loss: 9.6170 - accuracy: 0.7873


In [16]:
test_dataset = dataset.skip(10000)

In [17]:
batch_size = 10
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [18]:
# 保存模型
test_loss, test_acc = model.evaluate(test_dataset, verbose = 1)
model_filename = 'models/DenseNet121-dogsCNNModel1-' + "%.6f-"%(test_loss) + "%.6f"%(test_acc) +'.h5'
model.save_weights(model_filename)
print("save ok!")

   1058/Unknown - 1590s 2s/step - loss: 13.7676 - accuracy: 0.7521save ok!


In [ ]:
#模型结构保存
yaml_string = model.to_yaml()
with open('./models/DenseNet121-dogs.yaml', 'w') as model_file:
    model_file.write(yaml_string)
    

In [ ]:
from PIL import Image
# 加载图片
img = Image.open("dog1.jpg")

img_r = img.resize((224, 224))
img_data = np.array(img_r)
img_data2 = np.array([img_data])

plt.imshow(img_data)
plt.show()

print(img_data2.shape)

In [ ]:
# 进行预测
preds = model.predict(img_data2)

In [ ]:
preds

In [ ]:
def getName(i):
    dir_path = './images'
    f_l = [fn2 for fn2 in os.listdir(dir_path)]
    return f_l[i]
    

In [ ]:
i = tf.argmax(preds, 1).numpy()[0]

In [ ]:
f_l = [fn2 for fn2 in os.listdir(dir_path)]

In [ ]:
f_l[i]

In [ ]:
getName(90)